# Problem 2
### (a)

In [1]:
import numpy as np

A = np.array([
    [ 2, -1,  0,  0,  0,  0,  0,  0],
    [-1,  2, -1,  0,  0,  0,  0,  0],
    [ 0, -1,  2, -1,  0,  0,  0,  0],
    [ 0,  0, -1,  2, -1,  0,  0,  0],
    [ 0,  0,  0, -1,  2, -1,  0,  0],
    [ 0,  0,  0,  0, -1,  2, -1,  0],
    [ 0,  0,  0,  0,  0, -1,  2, -1],
    [ 0,  0,  0,  0,  0,  0, -1,  2]
], dtype=float)

def max_offdiag_index(A):
    n = A.shape[0]
    p, q = 0, 1
    m = abs(A[p, q])
    for i in range(n):
        for j in range(i+1, n):
            if abs(A[i, j]) > m:
                m = abs(A[i, j])
                p, q = i, j
    return p, q, m

def jacobi_cs(a_pp, a_qq, a_pq):
    if a_pq == 0.0:
        return 1.0, 0.0
    phi = 0.5 * np.arctan2(2.0 * a_pq, a_qq - a_pp)
    c = np.cos(phi)
    s = np.sin(phi)
    return c, s

def apply_jacobi_rotation(A, V, p, q):
    a_pp, a_qq, a_pq = A[p, p], A[q, q], A[p, q]
    c, s = jacobi_cs(a_pp, a_qq, a_pq)
    if s == 0:
        return
    n = A.shape[0]
    for i in range(n):
        if i != p and i != q:
            a_ip = A[i, p]
            a_iq = A[i, q]
            A[i, p] = c*a_ip - s*a_iq
            A[p, i] = A[i, p]
            A[i, q] = c*a_iq + s*a_ip
            A[q, i] = A[i, q]
    app, aqq, apq = a_pp, a_qq, a_pq
    A[p, p] = c*c*app - 2*s*c*apq + s*s*aqq
    A[q, q] = s*s*app + 2*s*c*apq + c*c*aqq
    A[p, q] = 0
    A[q, p] = 0
    for i in range(n):
        v_ip = V[i, p]
        v_iq = V[i, q]
        V[i, p] = c*v_ip - s*v_iq
        V[i, q] = s*v_ip + c*v_iq

def jacobi_eigenvalues(A_init, tol=1e-2, max_iterations=200000):
    A = A_init.copy().astype(float)
    n = A.shape[0]
    V = np.eye(n)
    k = 0
    while True:
        p, q, m = max_offdiag_index(A)
        if m < tol or k >= max_iterations:
            break
        apply_jacobi_rotation(A, V, p, q)
        k += 1
    return np.diag(A), V, k

eigvals, eigvecs, iters = jacobi_eigenvalues(A, tol=1e-2)

print("Approximate eigenvalues:")
print(eigvals)
print("\nNumber of Jacobi iterations:")
print(iters)

Approximate eigenvalues:
[0.12079488 0.46780265 1.00012657 1.65256292 2.34730234 2.99994978
 3.53209228 3.87936857]

Number of Jacobi iterations:
48


### (b)

In [2]:
import numpy as np

n = 8
k = np.arange(1, n+1)
lambda_exact = 2 - 2*np.cos(np.pi * k / (n+1))
print("Exact Eigenvalues:")
print(lambda_exact)

errors = eigvals - lambda_exact
print("\nErrors:")
print(errors)

Exact Eigenvalues:
[0.12061476 0.46791111 1.         1.65270364 2.34729636 3.
 3.53208889 3.87938524]

Errors:
[ 1.80125745e-04 -1.08465319e-04  1.26573492e-04 -1.40726113e-04
  5.98915815e-06 -5.02200681e-05  3.39497972e-06 -1.66718752e-05]


# Problem 3


In [ ]:
def householder_tridiagonal(A_init):
    A = A_init.copy().astype(float)
    n = A.shape[0]
    Q = np.eye(n)
    for k in range(n-2):
        x = A[k+1:, k]
        normx = np.linalg.norm(x)
        if normx == 0:
            continue
        sign = 1.0 if x[0] >= 0 else -1.0
        u1 = x[0] + sign*normx
        v = x.copy()
        v[0] = u1
        v = v / np.linalg.norm(v)
        A[k+1:, k:] -= 2.0 * np.outer(v, v @ A[k+1:, k:])
        A[:, k+1:] -= 2.0 * np.outer(A[:, k+1:] @ v, v)
        Q[:, k+1:] -= 2.0 * np.outer(Q[:, k+1:] @ v, v)
    return A, Q

def max_offdiag_index(A):
    n = A.shape[0]
    p, q = 0, 1
    m = abs(A[p, q])
    for i in range(n):
        for j in range(i+1, n):
            if abs(A[i, j]) > m:
                m = abs(A[i, j])
                p, q = i, j
    return p, q, m

def jacobi_cs(a_pp, a_qq, a_pq):
    if a_pq == 0.0:
        return 1.0, 0.0
    phi = 0.5 * np.arctan2(2.0 * a_pq, a_qq - a_pp)
    c = np.cos(phi)
    s = np.sin(phi)
    return c, s

def apply_jacobi_rotation(A, V, p, q):
    a_pp, a_qq, a_pq = A[p, p], A[q, q], A[p, q]
    c, s = jacobi_cs(a_pp, a_qq, a_pq)
    if s == 0:
        return
    n = A.shape[0]
    for i in range(n):
        if i != p and i != q:
            a_ip = A[i, p]
            a_iq = A[i, q]
            A[i, p] = c*a_ip - s*a_iq
            A[p, i] = A[i, p]
            A[i, q] = c*a_iq + s*a_ip
            A[q, i] = A[i, q]
    app, aqq, apq = a_pp, a_qq, a_pq
    A[p, p] = c*c*app - 2*s*c*apq + s*s*aqq
    A[q, q] = s*s*app + 2*s*c*apq + c*c*aqq
    A[p, q] = 0.0
    A[q, p] = 0.0
    for i in range(n):
        v_ip = V[i, p]
        v_iq = V[i, q]
        V[i, p] = c*v_ip - s*v_iq
        V[i, q] = s*v_ip + c*v_iq

def jacobi_after_householder(A_init, tol=1e-2, max_iterations=200000):
    T, Qh = householder_tridiagonal(A_init)
    A = T.copy()
    n = A.shape[0]
    V = np.eye(n)
    k = 0
    while True:
        p, q, m = max_offdiag_index(A)
        if m < tol or k >= max_iterations:
            break
        apply_jacobi_rotation(A, V, p, q)
        k += 1
    eigvals = np.diag(A)
    eigvecs = Qh @ V
    return eigvals, eigvecs, k

eigvals_hh_jacobi, eigvecs_hh_jacobi, iters_hh = jacobi_after_householder(A, tol=1e-2)

print("Approximate eigenvalues after Householder + Jacobi:")
print(eigvals_hh_jacobi)
print("\nNumber of Jacobi iterations:")
print(iters_hh)

Approximate eigenvalues after Householder + Jacobi:
[0.12079488 0.46780265 1.00012657 1.65256292 2.34730234 2.99994978
 3.53209228 3.87936857]

Number of Jacobi iterations (after tridiagonalization):
48
